# Stock Data & Sentiment Analysis Agent with Agno

This is an example of Stock Data & Sentiment Analysis Agent. It is used to demonstrate the creation of AI Agents that are autonomously driven for specific tasks that can follow rules and adapt to input. In this case, we focus on tasks such as testing and iterating different tool combination including Google search + yfinance.

For this example we leverage:

- Agno, a modern agent framework that’s actually built for tools and performance. It is model-agnostic but in this example we leverage a multi-model open weight model provided on Scaleway cloud infrastructure. Also the framework supports custom Python tools, which means we can easily extend it in teh future with custom tools for stock data analysis for example.  
- yfinance: Used to fetch financial data from Yahoo Finance.
- Google Search: Used to search the web for news articles.

## Execution Variables Setup

This section includes all the necessary variables to the execution of the data pipeline and the agentic flow. We also perform a quick test against the model endpoint with a simple agno agent before proceeding.

In [8]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv
load_dotenv()

# Environment variables for Google GenAI API inference
google_api_key = os.getenv("GOOGLE_API_KEY", "")
google_model = "gemini-2.5-flash"
google_model_eval = "gemini-2.5-pro"

# Environment variables for Scaleway GenAI API inference
scw_project_id = os.getenv("SCW_DEFAULT_PROJECT_ID", "")
scw_url = "https://api.scaleway.ai/" + scw_project_id + "/v1"
scw_api_key = os.getenv("SCW_SECRET_KEY", "")
scw_model = "qwen3-235b-a22b-instruct-2507"

In [9]:
from os import getenv
from agno.agent import Agent
from agno.models.openai.like import OpenAILike

agent = Agent(
    model=OpenAILike(
        id=scw_model,
        api_key=scw_api_key,
        base_url=scw_url,
    ),
    markdown=True,
    stream=True,
)

# Print the response in the terminal
agent.print_response("Share a 2 sentence horror story.")

Output()

In [10]:
# Sentiment Analysis Agent
# Introduces tools for web search and financial information retrieval
from agno.tools.googlesearch import GoogleSearchTools
from agno.tools.yfinance import YFinanceTools
sentiment_agent = Agent(
    name="Sentiment Analysis Agent",
    role="Search and interpret news articles",
    model=OpenAILike(
        id=scw_model,
        api_key=scw_api_key,
        base_url=scw_url,
    ),
    tools=[
        GoogleSearchTools(),
        YFinanceTools(include_tools=["get_company_info","get_company_news"]),
    ],
    instructions=[
        "Find relevant news articles for each company and critically analyze the news sentiment.",
        "Provide sentiment scores from 1 (negative) to 10 (positive) with reasoning and sources."
        "Cite your sources. Be specific, crtical and provide relevant links."
    ],
    markdown=True,
    stream=True,
)

In [11]:
from typing import Iterator
from agno.agent import RunOutputEvent
from agno.utils.pprint import pprint_run_response

# Run agent and return the response as a stream
response_stream: Iterator[RunOutputEvent] = sentiment_agent.run(
    "Analyze the sentiment for the following companies for the month of September 2025 (your context is up-to-date as of Sept 19 2025): NVDA, META. \n\n"
    "Search for relevant news articles and interpret the sentiment for each company. Provide the sentiment score.",
    stream=True,
    stream_intermediate_steps=True,
)

# Print the response stream in markdown format
pprint_run_response(response_stream, markdown=True)

Output()

In [12]:
# Financial Analyst Agent
basic_stock_agent = Agent(
    name="Basic Financial Data Agent",
    role="Retrieve basic company financial data and expertly interpret trends and data using a rigourously analytical approach",
    model=OpenAILike(
        id=scw_model,
        api_key=scw_api_key,
        base_url=scw_url,
    ),
    tools=[YFinanceTools(include_tools=["get_current_stock_price", 
                                        "get_historical_stock_prices",
                                        "get_company_info",
                                        "get_stock_fundamentals", 
                                        "get_analyst_recommendations"])],
    instructions=[
        "Retrieve stock prices, analyst recommendations, and key summary financial data.",
        "Focus on company funamentals and trends, presenting the data in tables with key insights."
    ],
    markdown=True,
    stream=True,
)

In [13]:
# Historical Data Analyst Agent
historical_financials_agent = Agent(
    name="Comprehensive Financial Data Agent",
    role="Retrieve comprehensive company financial data and expertly interpret trends and data using a rigourously analytical approach",
    model=OpenAILike(
        id=scw_model,
        api_key=scw_api_key,
        base_url=scw_url,
    ),
     tools=[YFinanceTools(include_tools=["get_company_info", 
                                        "get_key_financial_ratios",
                                        "get_income_statements", 
                                        "get_historical_stock_prices"])],
    instructions=[
        "Retrieve key financial ratios, income statement, balance sheet, cash flow statements and historical price data.",
        "Focus on company fundamentals and trends, presenting the data in tables with key insights."
    ],
    markdown=True,
    stream=True,
)

In [14]:
# Data Synthesis / Executive Agent
executive_agent = Agent(
    name="Executive Agent",
    role="You are a seasoned portfolio manager and executive. You are tasked with making data-driven decisions and drawing conclusions through critical assessment",
    model=OpenAILike(
        id=scw_model,
        api_key=scw_api_key,
        base_url=scw_url,
    ),
    instructions=[
        "Check outputs for accuracy and completeness.",
        "Synthesize data to provide a final sentiment score (1-10) with justifcation."
        "Finally, after critical reasoning, provide a BUY, SELL or HOLD recommendation for the stock."
    ],
    markdown=True,
    stream=True,
)

## Create the Multi Agent Orchestration (Build Team)

These few lines of code set up a sophisticated multi agent system with specific roles that collaborate to analyze a stock.

The `agent_team` acts as a conductor, picking the right agent work flow and ensuring that each agent contributes its insights and that the final output is well-structured, data-driven, and easily understandable

In [15]:
# Full Team of Agents
from agno.team import Team
from datetime import date

today = date.today().strftime("%Y-%m-%d")

team = Team(
    model=OpenAILike(
        id=scw_model,
        api_key=scw_api_key,
        base_url=scw_url,
    ),
    members=[sentiment_agent, basic_stock_agent, historical_financials_agent, executive_agent],
    instructions=[
        "Combine the expertise of all the agents part of your team to provide a cohesive, highly analytical and well-supported response",
        f"Always include references and dates for all data points and sources. Your context is current as of {today}.",
        "Present data in structured tables for clarity, ensuring all conclusions are data-driven.",
        "Explain the methodlogy and always determine a BUY, SELL, or HOLD recommendation.",
    ],
    markdown=True,
    stream=True,
    debug_mode=True,   # Set to True to view the detailed logs and see the compiled system message
)

In [17]:
# Asynchronous execution
for chunk in team.run(
    "Analyze the sentiment for the following companies for the month of September 2025 (your context is up-to-date as of Sept 19 2025): NVDA, META. \n\n"
    "1. **Sentiment Analysis**: Search for relevant news articles and interpret the sentiment for each company. Provide the sentiment score"
    "2. **Financial Data**: Analyze stock price and company fundamental financial metrics and other resources. Highlight key trends"
    "3. **Consolidated Analysis**: Combine the insights from the various srouces such as sentiment analysis and financial data to assign a final BUY, SELL or HOLD recommendation"
    "Ensure your response is accurate, comprehensive, analytical and well-reasoned. Include references to sources with publication dates.",
    stream=True,
    stream_intermediate_steps=True
):
    print(chunk.content, end="", flush=True)

DEBUG ***** Session ID: 63cdf5a7-697f-4543-8598-4b8704cc6b64 *****

DEBUG ****** Team ID: 3081995d-1906-4042-bfb3-e7a1bbb066bd *******

DEBUG Creating new TeamSession: 63cdf5a7-697f-4543-8598-4b8704cc6b64

DEBUG Processing tools for model

DEBUG Added tool delegate_task_to_member

...

DEBUG **********************  TOOL METRICS  **********************

DEBUG * Duration:                    0.2798s

DEBUG **********************  TOOL METRICS  **********************

ERROR    Rate limit error from OpenAI API: Error code: 429 - {'error': 'TOO MANY TOKENS', 'message': 'You exceeded 
         your current quota of tokens per minute', 'status': 429}

ModelProviderError: TOO MANY TOKENS